#Find DEGs with pseudobulk (limma) in the merge object Alkon + Reynolds
Done adjusting dataset as a covariable

###Relevant cell types (celltypist): 
T-cells (Tc, Th and Treg), Keratinocytes (Undifferentiated and Differentiated), Macrophages

###Constrast: 
  - Lesional vs Healthy control (LvsHC)

In [0]:
#Load required libraries
.libPaths(c("/dbfs/home/jtrincado@almirall.com/my_r_packages/Seurat", .libPaths()))
library(openxlsx)
library(dplyr)
.libPaths(c("/dbfs/home/pdelgadom@almirall.com/my_r_packages/tfm_paula_4", .libPaths()))
library(limma)
library(edgeR)

.libPaths(c("/dbfs/home/boriol@almirall.com/my_r_packages/bulkRNASeq_PBMCs_R4.3", .libPaths()))
library(VennDiagram)
library(EnhancedVolcano)

#Load required libraries
.libPaths(c("/dbfs/home/jtrincado@almirall.com/my_r_packages/Seurat_v2/", .libPaths()))
library(Seurat)

In [0]:
volcano_generator <- function(resultsDE, given_title) {
  library(dplyr)
  library(EnhancedVolcano)

  resultsDE <- as.data.frame(resultsDE)
  
  # Create annotations for volcano plot
  resultsDE0 <- resultsDE
  resultsDE0$gene_id <- rownames(resultsDE0)

  # Ensure unique row names and remove rows with missing gene id
  resultsDE0 <- resultsDE0 %>%
    distinct(gene_id, .keep_all = TRUE)
  rownames(resultsDE0) <- resultsDE0$gene_id
  
  # Determine column names for p-value and log2 fold change
  p_val_col <- if ("adj.P.Val" %in% colnames(resultsDE0)) "adj.P.Val" else "padj"
  log2fc_col <- if ("logFC" %in% colnames(resultsDE0)) "logFC" else "log2FoldChange"
  
  top10_genes <- resultsDE0 %>%
    filter(!!sym(log2fc_col) > 1 & !!sym(p_val_col) < 0.05) %>%
    arrange(!!sym(p_val_col)) %>% top_n(10, -!!sym(p_val_col))
  
  bottom10_genes <- resultsDE0 %>%
    filter(!!sym(log2fc_col) < -1 & !!sym(p_val_col) < 0.05) %>%
    arrange(!!sym(p_val_col)) %>% top_n(10, -!!sym(p_val_col))
  
  # Plot Volcano with overlap of labels set to infinite
  volcano <- EnhancedVolcano(resultsDE0,
    lab = rownames(resultsDE0),
    x = log2fc_col,
    y = p_val_col,
    pCutoff = 0.05,
    selectLab = c(top10_genes$gene_id, bottom10_genes$gene_id),
    labSize = 5,
    drawConnectors = TRUE,
    widthConnectors = 0.5,
    colConnectors = 'black',
    title = given_title,
    max.overlaps = Inf  # <- Overlap of labels set to infinite
  )
  volcano
}

In [0]:
#New volcano generator function to print a zoom in -10,10 and avoid extreme values
volcano_generator_zoom <- function(resultsDE, given_title) {
  library(dplyr)
  library(EnhancedVolcano)
  resultsDE <- as.data.frame(resultsDE)
  
  # Create annotations for volcano plot
  resultsDE0 <- resultsDE
  resultsDE0$gene_id <- rownames(resultsDE0)
  
  # Ensure unique row names and remove rows with missing gene id
  resultsDE0 <- resultsDE0 %>%
    distinct(gene_id, .keep_all = TRUE)
  rownames(resultsDE0) <- resultsDE0$gene_id
  
  # Determine column names for p-value and log2 fold change
  p_val_col <- if ("p_val_adj" %in% colnames(resultsDE0)) "p_val_adj" else "padj"
  log2fc_col <- if ("avg_log2FC" %in% colnames(resultsDE0)) "avg_log2FC" else "log2FoldChange"
  
  # Top and bottom genes for labeling
  top10_genes <- resultsDE0 %>%
    filter(!!sym(log2fc_col) > 1 & !!sym(p_val_col) < 0.05) %>%
    arrange(!!sym(p_val_col)) %>% top_n(10, -!!sym(p_val_col))
  
  bottom10_genes <- resultsDE0 %>%
    filter(!!sym(log2fc_col) < -1 & !!sym(p_val_col) < 0.05) %>%
    arrange(!!sym(p_val_col)) %>% top_n(10, -!!sym(p_val_col))
  
  # Plot Volcano with xlim for log2FC zoom
  volcano <- EnhancedVolcano(resultsDE0,
    lab = rownames(resultsDE0),
    x = log2fc_col,
    y = p_val_col,
    pCutoff = 0.05,
    selectLab = c(top10_genes$gene_id, bottom10_genes$gene_id),
    labSize = 5,
    drawConnectors = TRUE,
    widthConnectors = 0.5,
    colConnectors = 'black',
    title = given_title,
    xlim = c(-10, 10)  # <- ZOOM in here
  )
  
  return(volcano)
}

##Process data

In [0]:
AR <- readRDS(file="/dbfs/mnt/sandbox/TFM_PAULA/MERGED_ARdatasets_celltypist_TFM.rds")

In [0]:
unique(AR$dataset)

In [0]:
unique(AR$Condition_AR)

In [0]:
table(AR$celltype_AR, AR$dataset)

##Filtering variables that have at least 3 counts

In [0]:
counts_matrix <- AR[["RNA"]]$counts
dim(counts_matrix) #38248 391832

In [0]:
# Keep only rows that have a count of at least 3 counts in 3 samples
smallestGroupSize <- 3
keep <- rowSums(counts_matrix >= 3) >= smallestGroupSize
counts_keep <- counts_matrix[keep,]

# Subset the Seurat object to keep only the features in counts_keep
AR_f <- subset(AR, features = rownames(counts_keep))

# Assign the filtered counts to the new Seurat object
AR_f[["RNA"]]$counts <- counts_keep

# Check dimensions
dim(AR_f[["RNA"]]$counts) # 16417 391832

##Pseudobulk the counts based on the donor id

In [0]:
unique(AR_f$celltype_AR)

In [0]:
table(AR_f$Condition_AR, AR_f$Sample_id)

In [0]:
unique(AR_f$dataset)

In [0]:
# pseudobulk the counts based on donor-condition-celltype
pseudo_AR <- AggregateExpression(AR_f, assays = "RNA", return.seurat = T, group.by = c("Condition_AR", "Sample_id", "celltype_AR"))

# each 'cell' is a donor-condition-celltype pseudobulk profile
head(Cells(pseudo_AR))

In [0]:
pseudo_AR$celltype.cond <- paste(pseudo_AR$celltype_AR, pseudo_AR$Condition_AR, sep = "_")

In [0]:
Idents(pseudo_AR) <- "celltype.cond"

In [0]:
dataset_info <- AR_f@meta.data$dataset

# Create a mapping of Sample_id to dataset
sample_to_dataset <- setNames(dataset_info, AR_f@meta.data$Sample_id)

# Add the dataset information to the pseudo_AR metadata
pseudo_AR@meta.data$dataset <- sample_to_dataset[pseudo_AR@meta.data$Sample_id]

In [0]:
unique(pseudo_AR@meta.data$dataset)

In [0]:
saveRDS(pseudo_AR, file="/dbfs/mnt/sandbox/TFM_PAULA/AR_MERGED_aggregated_expression_TFM.rds")

In [0]:
# Extract count data
counts_AR <- GetAssayData(pseudo_AR, layer = "counts")

# Extract metadata
metadata_AR <- pseudo_AR@meta.data

In [0]:
# Filter to remove non lesional samples 

In [0]:
metadata_AR_reynolds <- metadata_AR[metadata_AR$Condition_AR %in% c("HC", "Lesional") & metadata_AR$dataset == "reynolds", ]
metadata_AR <- rbind(metadata_AR[metadata_AR$dataset != "reynolds", ], metadata_AR_reynolds)

# Filter counts too
counts_AR <- counts_AR[, colnames(counts_AR) %in% rownames(metadata_AR)]

In [0]:
metadata_AR$celltype.cond <- as.factor(metadata_AR$celltype.cond)
metadata_AR$dataset <- as.factor(metadata_AR$dataset)

In [0]:
table(metadata_AR$Condition_AR, metadata_AR$dataset)

In [0]:
# Ensure the same order for rows in metadata_AR and columns in counts_AR
metadata_AR <- metadata_AR[order(rownames(metadata_AR)), ]
counts_AR <- counts_AR[, order(colnames(counts_AR))]

# Reorder counts_AR columns to match the order of metadata_AR rows
counts_AR <- counts_AR[, rownames(metadata_AR)]

###Limma design to adjust the covariable dataset

In [0]:
unique(metadata_AR$Condition_AR)

In [0]:
metadata_AR$samples <- paste(metadata_AR$Condition_AR, metadata_AR$Sample_id, metadata_AR$celltype_AR, sep = "_")

In [0]:
metadata_AR$samples

In [0]:
library(edgeR)
library(limma)
library(dplyr)
 
wanted_celltypes <- c("Tc", "Th", "Differentiated-KC", "Undifferentiated-KC")
 
# expr_matrix = counts_AR
# metadata_AR must include:
#   - sample_id (matching colnames(counts_AR))
#   - celltype (for subsetting)
#   - condition (or celltype.cond)
#   - dataset (for batch)
 
# Sanity check: sample IDs match
stopifnot(all(colnames(counts_AR) %in% metadata_AR$samples))

# Optional: reorder metadata to match expression matrix
metadata_AR <- metadata_AR[match(colnames(counts_AR), metadata_AR$samples), ]
 
  # Subset expression matrix to relevant samples
  expr_ct <- counts_AR[, metadata_AR$samples, drop = FALSE]
  expr_ct <- as.matrix(expr_ct)
  mode(expr_ct) <- "numeric"
 
  # limma-voom pipeline
  dge_ct <- DGEList(counts = expr_ct)
  keep <- filterByExpr(dge_ct, group = metadata_AR$celltype.cond)
  dge_ct <- dge_ct[keep, , keep.lib.sizes = FALSE]
  dge_ct <- calcNormFactors(dge_ct)
 
  # Design matrix (batch + condition)
  design_ct <- model.matrix(~ 0 + dataset + celltype.cond, data = metadata_AR)
 
  v_ct <- voom(dge_ct, design_ct, plot = FALSE)
  fit_ct <- lmFit(v_ct, design_ct)
  fit_ct <- eBayes(fit_ct)
 


In [0]:
fit_ct

In [0]:
results_list <- list()
for (ct in wanted_celltypes){
  group_HC <- paste0("celltype.cond", ct, "_HC")
  group_LES <- paste0("celltype.cond", ct, "_Lesional")
  if (!(group_HC %in% colnames(design_ct)) || !(group_LES %in% colnames(design_ct))){
    cat("Skipping", ct, "- missing group\n")
    next
  }
  # Create contrast
  contrast_vec <- rep(0, ncol(design_ct))
  names(contrast_vec) <- colnames(design_ct)
  contrast_vec[group_LES] <- 1
  contrast_vec[group_HC] <- -1
  # Apply contrast
  fit2 <- contrasts.fit(fit_ct, contrast_vec)
  fit2 <- eBayes(fit2)
  # Extract DE results
  res <- topTable(fit2, coef = 1, number = Inf)
  res$celltype <- ct
  results_list[[ct]] <- res
  cat("Finished:", ct, "\n")
}
# Combine all into one dataframe
all_results <- bind_rows(results_list)
head(all_results)

##Tc

In [0]:
TC_results <- all_results %>% filter(celltype=="Tc")

In [0]:
TC_results$gene <- rownames(TC_results)

In [0]:
TC_results$logFC

In [0]:
class(TC_results$adj.P.Val)

In [0]:
sum(is.na(TC_results$logFC))

In [0]:
volcano_generator(TC_results, "Tcells - AR - Limma Merged Cov Adjusted")

##Th

In [0]:
Th_results <- all_results %>% filter(celltype=="Th")

In [0]:
Th_results$gene <- rownames(Th_results)

In [0]:
volcano_generator(Th_results, "Th - AR - Limma Merged Cov Adjusted")

##Differentiated Keratinocytes

In [0]:
DifKC_results <- all_results %>% filter(celltype=="Differentiated-KC")

In [0]:
DifKC_results$gene <- rownames(DifKC_results)

In [0]:
volcano_generator(DifKC_results, "Diff KC - AR - Limma Merged Cov Adjusted")

##Undifferentiated KC

In [0]:
UndifKC_results <- all_results %>% filter(celltype=="Undifferentiated-KC")

In [0]:
UndifKC_results$gene <- rownames(UndifKC_results)

In [0]:
volcano_generator(UndifKC_results, "Undiff KC - AR - Limma Merged Cov Adjusted")

#Save

In [0]:
%sh
mkdir /dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/DEGs/Limma

In [0]:
# Write the data frame to new Excel files
write.xlsx(TC_results, "/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/DEGs/Limma/alkon_limma_results_Tc.xlsx")
write.xlsx(UndifKC_results, "/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/DEGs/Limma/alkon_limma_results_UndifKC.xlsx")
write.xlsx(DifKC_results, "/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/DEGs/Limma/alkon_limma_results_DifKC.xlsx")
write.xlsx(Th_results, "/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/DEGs/Limma/alkon_limma_results_Th.xlsx")